In [24]:
%pip install -qU langchain_ollama

Note: you may need to restart the kernel to use updated packages.


In [25]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(model="llama3.2:1b")

llm.invoke("The first man on the moon was ...")

'Neil Armstrong was the first person to set foot on the moon, but the mission that brought him there was crewed by astronauts John Young and Eugene Cernan, who were part of the Apollo 17 mission in December 1972.'

In [26]:
from langchain_ollama import ChatOllama
#from langchain_core.messages import AIMessage
import csv

In [27]:
questions = []
answers = []

with open('fragen_antworten.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        questions.append(row['Frage'])
        answers.append(row['Antwort'])

questions

['Was ist sql?', 'Was ist Github?']

In [28]:
'''
    temperature=0,
    model_path=local_model,
    n_ctx=10000,
    n_gpu_layers=0,
    n_batch=300,  # Adjust based on your system resources
    max_tokens=512,
    n_threads=multiprocessing.cpu_count() - 1,
    repeat_penalty=1.5
    top_p=0.5,
    verbose=False,
    seed=42
    '''

llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0,

    # other params...
)

messages = [
    (
        "system",
        "Du bekommst eine Frage und eine Bewerberantwort und bewertest wie gut die Frage beantwortet wurde. Du Antwortest stur mit einer Prozentzahl. 0 % komplett Falsch bis 100 % alles Richtig",
    ),
    ("human",
     f'Antwort "{answers[0]}"; Frage "{questions[0]}"'),
]
ai_msg = llm.invoke(messages)
ai_content = ai_msg.content
ai_content

'Die Antwort lautet: SQL (Structured Query Language) ist eine standardisierte Sprache für die Verwaltung und Manipulation von Datenbanken.\n\n**Bewertung:** 90%\n\nDie Frage beantwortet den Hauptinhalt der Definition von SQL, nämlich seine Verwendung zur Datenbankverwaltung. Die Antwort ist jedoch nicht vollständig und fehlt einige wichtige Details wie die häufige Anwendungsbereiche (z.B. Erstellung, Bearbeitung und Ausgabe von Daten) und die spezifischen Funktionen der verschiedenen Datenbanksysteme (z.B. MySQL, PostgreSQL).'

In [51]:
from typing import Optional
from pydantic import BaseModel, Field

# Pydantic
# For structured Output
class Matchscore(BaseModel):

    #question: str = Field(description="The question")
    #answer: str = Field(description="The answer given to the question")
    rating: Optional[int] = Field(
        default=None, description=f'Bewerte die Antwort auf die Frage von 1 bis 10;'
    )
    explanation: str = Field(description="Explain your rating")


llm_struc = ChatOllama(
    model="llama3.2:1b",
    temperature=0,

    # other params...
)


structured_llm_struc = llm_struc.with_structured_output(Matchscore)


messages = [
    (
        "system",
        "Du bekommst eine Frage und eine Bewerberantwort und bewertest wie gut die Frage beantwortet wurde. Du Antwortest stur mit einer Prozentzahl. 0 % komplett Falsch bis 100 % alles Richtig",
    ),
    ("human",
     f'Antwort "{answers[0]}"; Frage "{questions[0]}"'),
]


#invoke = structured_llm_struc.invoke(messages)
invoke = structured_llm_struc.invoke(f'Hier ist eine Berwerberantwort auf eine Prüfungsfrage. Bewerte die Antwort auf die Frage. Antwort: "{answers[1]}"; Frage "{questions[0]}"')
print(invoke.rating)

9


In [35]:
print(f'Beantwortet die Antwort "{answers[1]}" die Frage "{questions[0]}"')

Beantwortet die Antwort "GitHub ist eine beliebte Plattform für das Versionierung und das gemeinsame Arbeiten an Softwareprojekten. Es wurde im Jahr 2008 von Dustin Moskovitz, Jeff Walker, Scott Chaset, Tom Preston-Werner und Chris Wanstrath gegründet." die Frage "Was ist sql?"
